The purpose of autonomously adjusting reference voltage (AARV) is to mitigate rapid voltage change (RVC), without affecting the utility’s control of steady-state voltage or the DER’s steady-state reactive power.  AARV may be used with a volt-watt function that mitigates steady-state overvoltage, but otherwise AARV does not regulate steady-state voltage.

Page 39 of IEEE 1547-2018 says: _The DER shall be capable of autonomously adjusting reference voltage (VRef) with VRef being equal to the low pass filtered measured voltage. The time constant shall be adjustable at least over the range of 300 s to 5000 s. The voltage-reactive power Volt-Var curve characteristic shall be adjusted autonomously as VRef changes. The approval of the Area EPS operator shall be required for the DER to autonomously adjust the reference voltage. Implementation of the autonomous VRef adjustability and the associated time constant shall be specified by the Area EPS operator._

Clause 5.14.5 of IEEE 1547.1-2020 describes a test for AARV. In all other tests, AARV is to be disabled. Four AARV tests are described, at the time constant Tref = 300s and 5000s, each of those with source voltage steps to (V1+V2)/2 and (V3+V4)/2. Before each voltage step, the steady-state reactive power, Q, should be zero at V=Vref. The voltage steps result in step changes of Q to 0.5*Q1 and 0.5*Q4, respectively, with some delay from the open-loop response. These new Q values should follow an exponential decay back to zero, governed by Tref = 300s or 5000s. The test criteria specifies that after one time constant, Tref, the value of Q should be less than 10% of Q1 or Q4, respectively. However, the expected value of Q, following an exponential decay from the stepped value, is Q = 0.5*Q1*exp(-Tref/Tref) = 0.1839*Q1 or 0.1839*Q4. The evaluation criteria at 1*Tref should be 20% instead of 10%.  Note: at 2*Tref, the expected value of Q is 0.0677*Q1 or 0.0677*Q4. This test verifies implementation of Tref, but not the impact of AARV on a system with grid impedance.

2019 Conference Paper: https://doi.org/10.1109/PVSC40753.2019.8981277 

2023 Conference Paper: https://doi.org/10.1109/PESGM52003.2023.10252317

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import math
plt.rcParams['savefig.directory'] = os.getcwd()

Run the following cells to generate the plots using [Matplotlib](https://matplotlib.org/)

In [ ]:
dt = 0.1
Tref = 300.0
Tresponse = 5.0
TauOL = Tresponse / 2.3026
IncRef = 1.0 - math.exp(-dt/Tref)
IncOL = 1.0 - math.exp(-dt/TauOL)

Vreg = 1.0
dB = 0.0 # 0.02
K = 22.0
#Verr = Vpu - Vreg
#Vreg = Vreg + Verr * IncRef
#Qtarg = -K * (V - Vreg) + Qbias # differs from OpenDSS in that it does not iterate
#Q = Qlast + (Q - Qlast) * IncOL

# plot P, Q, V, Vreg, Vsrc
# input Vreg, K, dB, Q1, Q4
# calculate Q2 = Q3 = 0

def set_characteristic (center=1.0, deadband=0.0, slope=22.0, qmax=0.44, qmin=-0.44):
    Vreg = center
    Q1 = qmax
    Q2 = 0.0
    Q3 = 0.0
    Q4 = qmin
    V2 = center - 0.5 * deadband
    V3 = center + 0.5 * deadband
    V1 = V2 - Q1 / slope
    V4 = V3 - Q4 / slope
    VL = min(V1, 0.95)
    VH = max(V4, 1.05)
    VTABLE = np.array ([VL, V1, V2, V3, V4, VH])
    QTABLE = np.array ([Q1, Q1, Q2, Q3, Q4, Q4])
    return VTABLE, QTABLE
#print ('Q = [{:7.4f},{:7.4f},{:7.4f},{:7.4f}]'.format (Q1, Q2, Q3, Q4)) 
#print ('V = [{:7.4f},{:7.4f},{:7.4f},{:7.4f}]'.format (V1, V2, V3, V4))

VTABLE, QTABLE = set_characteristic (Vreg, dB, K, 0.44, -0.44)
VMIN = VTABLE[0]-0.01
VMAX = VTABLE[-1]+0.01

v = np.linspace (VMIN, VMAX, 501)
q = np.interp (v, VTABLE, QTABLE)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1, 1, sharex = 'col', figsize=(6,4), constrained_layout=True)
fig.suptitle ('Volt-var characteristic for Vreg={:.3f}, dB={:.3f}, K={:.2f}'.format (Vreg, dB, K))
ax.plot(VTABLE, QTABLE, marker='o', color='blue', label='Points')
ax.plot (v, q, color='red', label='Interpolated')
ax.grid()
ax.set_xlabel ('V [pu]')
ax.set_ylabel ('Q [pu]')
ax.set_xlim (VMIN, VMAX)
ax.legend()

plt.show()

In [ ]:
# approximate voltage change for the Nantucket BESS
Sbase = 6.0e6
Vbase = 13200.0
Zbase = Vbase*Vbase/Sbase
R = 1.210 / Zbase # 1.744
X = 2.8339 / Zbase # 2.6797
dP = 6.0e6 / Sbase
dQ = 0.0 / Sbase
Vs = 1.0

def get_voltage (rpu, xpu, dP, dQ, vspu):
    a1 = vspu*vspu + rpu*dP + xpu*dQ
    a2 = xpu*dP - rpu*dQ
    d = math.sqrt (a1*a1 + a2*a2) / vspu / vspu - 1.0
    vpu = vspu + d
    return vpu, d

vpoc, d = get_voltage (R, X, dP, dQ, Vs)

print ('Vpoc = {:.4f} pu, d = {:.4f} pu'.format (vpoc, d))

# Test

In [ ]:
# reset to the starting point
VTABLE, QTABLE = set_characteristic (Vreg, dB, K, 0.44, -0.44)

VTEST = [(VTABLE[1]+VTABLE[2])/2.0, (VTABLE[3]+VTABLE[4])/2.0]
QTEST = np.interp (VTEST, VTABLE, QTABLE)

fig, ax = plt.subplots(1, 2, sharex = 'col', figsize=(10,4), constrained_layout=True)
fig.suptitle ('AARV test for Tresponse={:.1f}, Tref={:.1f}'.format (Tresponse, Tref))

ax[0].set_title ('Characteristic')
ax[0].plot (VTABLE, QTABLE, marker='o', color='blue', label='Points')
ax[0].plot (VTEST, QTEST, color='red', marker='s', linestyle='None', label='Test')
ax[0].set_xlabel ('V [pu]')
ax[0].set_xlim (VMIN, VMAX)
ax[0].legend()

dt = 0.1
Tref = 300.0
Tresponse = 5.0
TauOL = Tresponse / 2.3026
IncRef = 1.0 - math.exp(-dt/Tref)
IncOL = 1.0 - math.exp(-dt/TauOL)
Tmax = 15.0 * Tref

t = np.linspace (0, Tmax, int(Tmax / dt) + 1)
npts = len(t)
v0 = (VTABLE[1]+VTABLE[2])/2.0
q0 = np.interp(v0, VTABLE, QTABLE)
q = np.zeros(npts)
vset = 1.0
vref = 1.0
Qmax = 0.0
#print ('       t      Vpu     Vreg     Verr    Qtarg        Q')
for i in range(1, npts):
    verr = v0 - vref
    vref = vref + verr * IncRef
    #qtarg = np.interp (v0 + (vset - vreg), VTABLE, QTABLE)
    VTABLE, QTABLE = set_characteristic (vref, deadband=dB, slope=K)
    qtarg = np.interp (v0, VTABLE, QTABLE)
    q[i] = q[i-1] + (qtarg - q[i-1]) * IncOL
    if abs(q[i]) > Qmax:
        Qmax = abs(q[i])
    #print ('{:8.3f} {:8.5f} {:8.5f} {:8.5f} {:8.5f} {:8.5f}'.format (t[i], v0, vref, verr, qtarg, q[i])) 

print ('{:d} test points, IncRef={:.6f}, IncOL={:.6f}, Qmax={:.3f}, Qend={:.3f}'.format (npts, IncRef, IncOL, Qmax, q[-1]))

ax[1].set_title ('Test Vs={:.3f}, Qs={:.2f} pu'.format (v0, q0))
ax[1].plot (t, q, color='red')
ax[1].set_xlabel ('t [s]')
ax[1].set_xlim (t[0], t[-1])

for i in range(2):
    ax[i].grid()
    ax[i].set_ylabel ('Q [pu]')

plt.show()

In [ ]:
# reset to the starting point
VTABLE, QTABLE = set_characteristic (Vreg, dB, K, 0.44, -0.44)

vsrc = np.ones(npts)
vpoc = np.ones(npts)
p = np.interp(t, [0.0, 0.2, 5.0, t[-1]], [0.0, 0.0, dP, dP])
q = np.zeros(npts)
vref = np.ones(npts)
verr = np.zeros(npts)
# this approach resets the V1..V4 points each time Vref changes
for i in range(npts-1):
    vpoc[i], d = get_voltage (R, X, p[i], q[i], vsrc[i])
    verr[i] = vpoc[i] - vref[i]
    vref[i+1] = vref[i] + verr[i] * IncRef
    VTABLE, QTABLE = set_characteristic (vref[i], deadband=dB, slope=K)
    qtarg = np.interp (vpoc[i], VTABLE, QTABLE)
    q[i+1] = q[i] + (qtarg - q[i]) * IncOL

print ('Ending Vref={:.4f}, IncRef={:.6f}, Maximum Verr={:.4f}'.format (vref[-1], IncRef, np.max(verr)))
fig, ax = plt.subplots(1, 2, sharex = 'col', figsize=(10,4), constrained_layout=True)
fig.suptitle ('AARV Response for dP={:.3f}, R={:.3f}, X={:.3f}'.format (dP, R, X))
ax[0].plot (t, p, label='P')
ax[0].plot (t, q, label='Q')
ax[1].plot (t, vsrc, label='Vsrc')
ax[1].plot (t, vpoc, label='Vpoc')
ax[1].plot (t, vref, label='Vref')
#ax[1].plot (t, verr, label='Verr')
for i in range(2):
    ax[i].set_ylabel ('[pu]')
    ax[i].set_xlabel ('[s]')
    ax[i].set_xlim (t[0], t[-1])
    ax[i].legend()
    ax[i].grid()
plt.show()